![MLU Logo](../../data/MLU_Logo.png)

# <a name="0">Responsible AI - Equalized Odds</a>

This notebook shows how to process outputs produced by a probabilistic model to generate fairer results. We will use a logistic regression model to predict whether an individuals' income is $\leq$ 50k or not using US census data.

__Dataset:__ 
The dataset we will use for this exercise is coming from [folktables](https://github.com/zykls/folktables). Folktables provide code to download data from the American Community Survey (ACS) Public Use Microdata Sample (PUMS) files managed by the US Census Bureau. The data itself is governed by the terms of use provided by the Census Bureau. For more information, see the [Terms of Service](https://www.census.gov/data/developers/about/terms-of-service.html).

__ML Problem:__ 
Ultimately, the goal will be to predict whether an individual's income is above \\$50,000. We will filter the ACS PUMS data sample to only include individuals above the age of 16, who reported usual working hours of at least 1 hour per week in the past year, and an income of at least \\$100. The threshold of \\$50,000 was chosen so that this dataset can serve as a comparable substitute to the [UCI Adult dataset](https://archive.ics.uci.edu/ml/datasets/adult). The income threshold can be changed easily to define new prediction tasks.


1. <a href="#1">Read the dataset</a>
2. <a href="#2">Data Processing</a>
    * <a href="#21">Exploratory Data Analysis</a>
    * <a href="#22">Select features to build the model</a>
    * <a href="#23">Feature Transformation</a>
    * <a href="#24">Train - Validation - Test Datasets</a>
    * <a href="#25">Data processing with Pipeline and ColumnTransformer</a>
3. <a href="#3">Train (and Tune) a Classifier</a>
4. <a href="#4">Test the Classifier</a>

This notebook assumes an installation of the SageMaker kernel `.conda-mlu-rai:Python` through the `environment.yaml` file in SageMaker Sudio Labs.

In [ ]:
%%capture

# Reshaping/basic libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

sns.set_style("darkgrid", {"axes.facecolor": ".9"})

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Operational libraries
import sys

sys.path.append("..")
sys.path.insert(1, "..")

# Fairness libraries
from folktables.acs import *
from folktables.folktables import *
from folktables.load_acs import *
from fairlearn.reductions import EqualizedOdds
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import MetricFrame, selection_rate

# Jupyter(lab) libraries
import warnings

warnings.filterwarnings("ignore")

## 1. <a name="1">Read the dataset</a>
(<a href="#0">Go to top</a>)

To read in the dataset, we will be using [folktables](https://github.com/zykls/folktables) which provides access to the US Census dataset. Folktables contains predefined prediction tasks but also allows the user to specify the problem type.

The US Census dataset distinguishes between household and individuals. To obtain data on individuals, we use `ACSDataSource` with `survey=person`. The feature names for the US Census data follow the same distinction and use `P` for `person` and `H` for `household`, e.g.: `AGEP` refers to age of an individual.

In [ ]:
income_features = [
    "AGEP",  # age individual
    "COW",  # class of worker
    "SCHL",  # educational attainment
    "MAR",  # marital status
    "OCCP",  # occupation
    "POBP",  # place of birth
    "RELP",  # relationship
    "WKHP",  # hours worked per week past 12 months
    "SEX",  # sex
    "RAC1P",  # recorded detailed race code
    "PWGTP",  # persons weight
    "GCL",  # grand parents living with grandchildren
]

# Define the prediction problem and features
ACSIncome = folktables.BasicProblem(
    features=income_features,
    target="PINCP",  # total persons income
    target_transform=lambda x: x > 50000,
    group="RAC1P",
    preprocess=adult_filter,  # applies the following conditions; ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    postprocess=lambda x: x,  # applies post processing, e.g. fill all NAs
)

# Initialize year, duration ("1-Year" or "5-Year") and granularity (household or person)
data_source = ACSDataSource(survey_year="2018", horizon="1-Year", survey="person")
# Specify region (here: California) and load data
ca_data = data_source.get_data(states=["CA"], download=True)
# Apply transformation as per problem statement above
ca_features, ca_labels, ca_group = ACSIncome.df_to_numpy(ca_data)

# Convert numpy array to dataframe
df = pd.DataFrame(
    np.concatenate((ca_features, ca_labels.reshape(-1, 1)), axis=1),
    columns=income_features + [">50k"],
)

# For further modelling you want to use only 2 groups
df = df[df["RAC1P"].isin([6, 8])].copy(deep=True)

## 2. <a name="2">Data Processing</a>
(<a href="#0">Go to top</a>)

### 2.1 <a name="21">Exploratory Data Analysis</a>
(<a href="#2">Go to Data Processing</a>)

We look at number of rows, columns, and some simple statistics of the dataset.

In [ ]:
# Print the first five rows
# NaN means missing data
df.head()

In [ ]:
# Check how many rows and columns we have in the data frame
print("The shape of the dataset is:", df.shape)

In [ ]:
# Let's see the data types and non-null values for each column
df.info()

We can clearly see that all columns are numerical (`dtype = float64`). However, when checking the column headers (and information at top of the notebook), we should notice that we are actually dealing with multimodal data. We expect to see a mix of categorical, numerical and potentially even text information.

Let's cast the features accordingly. We start by creating list for each feature type.

In [ ]:
categorical_features = [
    "COW",
    "SCHL",
    "MAR",
    "OCCP",
    "POBP",
    "RELP",
    "SEX",
    "GCL",
]

numerical_features = ["AGEP", "WKHP", "PWGTP"]

In [ ]:
# Cast categorical features to `category`
df[categorical_features] = df[categorical_features].astype("object")

# Cast numerical features to `int`
df[numerical_features] = df[numerical_features].astype("int")

Let's check with `.info()` again to make sure the changes took effect.

In [ ]:
df.info()

Looks good, so we can now separate model features from model target and sensitive feature to explore them separately.

In [ ]:
sensitive_feature = "RAC1P"

model_target = ">50k"
model_features = categorical_features + numerical_features

print("Model features: ", model_features)
print("Model target: ", model_target)

In [ ]:
# Double check that that target is not accidentally part of the features
model_target in model_features

All good here. We made sure that the target is not in the feature list. If we find the above statement showing `True` we need to remove the target by calling `model_features.remove(model_target)`.

Let's have a look at missing values next.


#### Missing values
The quickest way to check for missing values is to use `.isna().sum()`. This will provide a count of how many missing values we have. In fact, we can also see the count of missing values with `.info()` as it provided a count of non-null values.

In [ ]:
# Show missing values
df.isna().sum()

Before starting with the plots, let's have a look at how many unique instances we have per column. This helps us avoid plotting charts with hundreds of unique values. Let's filter for columns with fewer than 10 unique instances.

In [ ]:
shortlist_fts = (
    df[model_features]
    .apply(lambda col: col.nunique())
    .where(df[model_features].apply(lambda col: col.nunique()) < 10)
    .dropna()
)

print(shortlist_fts)

#### Target distribution

Let's check our target distribution.

In [ ]:
df[model_target].value_counts().plot.bar(color="black")
plt.show()

We notice that we are dealing with an imbalanced dataset. This means there are more examples for one type of results (here: 0; meaning individuals earning $\leq$ 50k). This is relevant for model choice and potential up-sampling or down-sampling to balance out the classes.

#### Feature distribution(s)

Let's now plot bar charts for the shortlist features of our dataset (as per above: shortlist - feature columns with less than 10 unique instance classes).

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
fig.suptitle("Feature Bar Plots")

fts = range(len(shortlist_fts.index.tolist()))
for i, ax in zip(fts, axs.ravel()):
    df[shortlist_fts.index.tolist()[i]].value_counts().plot.bar(color="black", ax=ax)
    ax.set_title(shortlist_fts.index.tolist()[i])
plt.show()

### 2.2 <a name="22">Select features to build the model</a>
(<a href="#2">Go to Data Processing</a>)

During the extended EDA in the DATAPREP notebook, we learned that `GCL` is a feature that is equally present for both outcome types and also contains a lot of missing values. Therefore, we can drop it from the list of features we want to use for model build. We also drop `OCCP` and `POBP` as those features have too many unique categories.

In [ ]:
to_remove = ["GCL", "OCCP", "POBP"]

# Drop to_remove features from the respective list(s) - if applicable
for ft in to_remove:
    if ft in model_features:
        model_features.remove(ft)
    if ft in categorical_features:
        categorical_features.remove(ft)
    if ft in numerical_features:
        numerical_features.remove(ft)

# Let's also clean up the dataframe and only keep the features and columns we need
df = df[model_features + [sensitive_feature] + [model_target]].copy(deep=True)

### 2.3 <a name="23">Feature transformation</a>
(<a href="#2">Go to Data Processing</a>)

In this notebook, we won't perform any transformation.

### 2.4 <a name="23">Train - Validation - Test Datasets</a>
(<a href="#2">Go to Data Processing</a>)

To get a training, test and validation set, we will use sklearn's [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

In [ ]:
train_data, test_data = train_test_split(
    df, test_size=0.1, shuffle=True, random_state=23
)

train_data, val_data = train_test_split(
    train_data, test_size=0.15, shuffle=True, random_state=23
)

# Print the shapes of the Train - Test Datasets
print(
    "Train - Test - Validation datasets shapes: ",
    train_data.shape,
    test_data.shape,
    val_data.shape,
)

### 2.4 <a name="24">Data processing with Pipeline and ColumnTransformer</a>
(<a href="#2">Go to Data Processing</a>)

Let's build a full model pipeline. We need preprocessing split per data type, and then combine everything back into a composite pipeline along with a model. To achieve this, we will use sklearns `Pipeline` and `ColumnTransformer`.

__Step 1 (set up pre-processing per data type):__
> For the numerical features pipeline, the __numerical_processor__ below, we impute missing values with the mean using sklearn's `SimpleImputer`, followed by a `MinMaxScaler` (don't have to scale features when using Decision Trees, but it's a good idea to see how to use more data transforms). If different processing is desired for different numerical features, different pipelines should be built - just like shown below for the two text features.

 > In the categorical features pipeline, the __categorical_processor__ below, we impute with a placeholder value and encode with sklearn's `OneHotEncoder`. If computing memory is an issue, it is a good idea to check categoricals' unique values, to get an estimate of many dummy features will be created by one-hot encoding. Note the __handle_unknown__ parameter that tells the encoder to ignore (rather than throw an error for) any unique value that might show in the validation/and or test set that was not present in the initial training set.
 
__Step 2 (combining pre-processing methods into a transformer):__ 
 > The selective preparations of the dataset features are then put together into a collective `ColumnTransformer`, to be finally used in a Pipeline along with an estimator. This ensures that the transforms are performed automatically on the raw data when fitting the model and when making predictions, such as when evaluating the model on a validation dataset via cross-validation or making predictions on a test dataset in the future.
   
__Step 3 (combining transformer with a model):__ 
> Combine `ColumnTransformer` from Step 2 with a selected algorithm in a new pipeline. For example, the algorithm could be a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for classification problems.

In [ ]:
### STEP 1 ###
##############

# Preprocess the numerical features
numerical_processor = Pipeline(
    [("num_imputer", SimpleImputer(strategy="mean")), ("num_scaler", MinMaxScaler())]
)
# Preprocess the categorical features
categorical_processor = Pipeline(
    [
        ("cat_imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("cat_encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

### STEP 2 ###
##############

# Combine all data preprocessors from above
data_processor = ColumnTransformer(
    [
        ("numerical_processing", numerical_processor, numerical_features),
        ("categorical_processing", categorical_processor, categorical_features),
    ]
)


### STEP 3 ###
##############

# Pipeline desired all data transformers, along with an estimator at the end
# Later you can set/reach the parameters using the names issued - for hyperparameter tuning, for example
pipeline = Pipeline(
    [
        ("data_processing", data_processor),
        ("lg", LogisticRegression(random_state=0)),
    ]
)

# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config

set_config(display="diagram")
pipeline

## 3. <a name="3">Train a Classifier</a>
(<a href="#0">Go to top</a>)

We use the pipeline, along with a Logistic Regression estimator for training and Equalized Odds postprocessing.

### Model Training

We train the classifier with __.fit()__ on our training dataset. 

In [ ]:
# Get train data to train the classifier
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the classifier to the train data
# Train data going through the Pipeline is imputed (with means from the train data),
#   scaled (with the min/max from the train data),
#   and finally used to fit the model
pipeline.fit(X_train, y_train)

y_train_pred = pipeline.predict(X_train)

Next, we want to enforce Equalized Odds. To do so, we are going to use fairlearns `ThresholdOptimzer` to implement Equality of Opportunity. It is a postprocessing algorithm based on the paper [Equality of Opportunity in Supervised Learning](https://proceedings.neurips.cc/paper/2016/file/9d2682367c3935defcb1f9e247a97c0d-Paper.pdf). This technique takes  an existing classifier and the sensitive feature as inputs, and derives a transformation of the classifier's prediction to enforce the specified parity constraints.

In [ ]:
# Set up ThresholdOptimizer
eo_model = ThresholdOptimizer(
    estimator=pipeline[-1],
    constraints="equalized_odds",
    objective="accuracy_score",
    grid_size=1000,
    flip=False,
    prefit=False,
    predict_method="deprecated",
)

# Learn the transformation & extract feature names
data_processor.fit(X_train)

# To extract feature names we first need to fit the data processor as this will generate the one hot encoding
ft_names = numerical_features + list(
    data_processor.transformers_[1][1]
    .named_steps["cat_encoder"]
    .get_feature_names_out(categorical_features)
)

# Add column names and convert to data frame
X_train_prep = pd.DataFrame(
        data_processor.transform(X_train).todense(), columns=ft_names
    )

# Adjust the results that the classifier would produce by letting ThresholdOptimizer know what the sensitive features are
eo_model.fit(X_train_prep, y_train, sensitive_features=train_data["RAC1P"].values)

In [ ]:
# You can now use the fitted equalized odds post-processor and use it to create adjusted outcomes (predictions)
y_train_adjusted = eo_model.predict(
    X_train_prep, sensitive_features=train_data["RAC1P"].values
)

In [ ]:
# Join the data, adjusted predictions, original predictions, and the true outcome
eop_df_train = pd.DataFrame(
    {
        "RAC1P": train_data["RAC1P"].reset_index(drop=True),
        "y_train_adjusted": y_train_adjusted,
        "y_train_pred": y_train_pred,
        "y_train_true": train_data[model_target].reset_index(drop=True),
    }
)

## 4. <a name="4">Test the Classifier</a>
(<a href="#0">Go to top</a>)

Let's now evaluate the performance of the trained classifier on the test dataset. We use __.predict()__ this time. 

In [ ]:
# Create the adjusted predictions for test and convert to dataframe
y_test_adjusted = eo_model.predict(
    np.asarray(data_processor.transform(test_data[model_features]).todense()),
    sensitive_features=test_data["RAC1P"].values,
)

# Let's have a look at the adjusted outputs for the test dataset
print(y_test_adjusted)

In [ ]:
# Join the data, adjusted predictions, original predictions, and the true outcome
eop_df_test = pd.DataFrame(
    {
        "RAC1P": test_data["RAC1P"].reset_index(drop=True),
        "y_test_adjusted": y_test_adjusted,
        "y_test_true": test_data[model_target].reset_index(drop=True),
        "y_test_withoutEO": pipeline.predict(test_data[model_features]),
    }
)

%matplotlib inline
# Initialize figure
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 6))

# Set title of figure
fig.suptitle("Comparison of Model Predictions and Baseline")

# Set title
ax1.title.set_text("Baseline target distribution")
ax2.title.set_text("Prediction EO adjusted")
ax3.title.set_text("Prediction without EO adjustment")

# Create plots
eop_df_test.groupby(["RAC1P", "y_test_true"]).size().unstack().plot(
    kind="bar", stacked=True, color=sns.husl_palette(2), ax=ax1
)
eop_df_test.groupby(["RAC1P", "y_test_adjusted"]).size().unstack().plot(
    kind="bar", stacked=True, color=sns.husl_palette(2), ax=ax2
)
eop_df_test.groupby(["RAC1P", "y_test_withoutEO"]).size().unstack().plot(
    kind="bar", stacked=True, color=sns.husl_palette(2), ax=ax3
)
# Align y-axis
ax2.sharey(ax1)
ax3.sharey(ax1)

You can clearly see that the ratios between the groups were adjusted drastically (compared to predictions without EO postprocessing). The question remains whether it was fair the reduce the number of positive outcomes for RAC1P group 6. The odds are equal now, but this came at the cost of placing a lot of outcomes that were actually positive into the negative bucket. You can see that if you hadn't enforced Equalized Odds, the model would have actually amplified the existing bias in the dataset.

This is the end of the notebook.